In [18]:
# Script merges all XLSX files which are in the same folder
# Creation date: 12 Oct 2022
# Last modification: 12 Oct 2022
#5531sek (92min) per 500 websites 

# Imports
import pandas as pd
import os
import glob
from urllib.parse import urlparse
  
path = os.getcwd() + "\\output" # Excel files must always be in "output" folder
excel_files = glob.glob(os.path.join(path, "*.xlsx"))

dataframes = []
domains = []
# loop over the list of excel_files
for f in excel_files:
    f_name = f.split("\\")[-1] # Take file name
    if f_name[:2] == "g_": # Intention is to only merge files that starts with "g_"
        dataframes.append(pd.read_excel(f))

try:
    merged_df = pd.concat(dataframes).drop_duplicates(subset=['Website']) # Merge all files
    merged_df = merged_df.drop(columns=["index", "Unnamed: 0"]) # Drop unnecessary comuns
    for website in merged_df["Website"]: # Loop to take only domain from URL
        domains.append(urlparse(website).netloc.replace("www.", "").lower())
    merged_df.insert(0, "Domain", domains) # Insert new column (loc, column_name, value)
    merged_df.insert(3, "Emails", "N/a")
    master_file = pd.read_excel(f'{path}\\master_file.xlsx')
    master_merged = pd.concat([master_file, merged_df]).drop_duplicates(subset=['Website'], keep='first') # Keep first records (the ones that were in master_file)
    master_merged.to_excel(f'{path}\\master_file.xlsx', index=False) # Save to Excel file
except Exception as e:
    print(e)
